In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')


from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset


C:\Users\roger\AppData\Local\Temp\ipykernel_5684\2816518301.py:8: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats('svg')


In [2]:
import seaborn as sns
import torch

# Iris ডেটাসেট লোড করো
iris = sns.load_dataset('iris')

# Features (4টি) কে float32 tensor বানাও
data = torch.tensor(iris[iris.columns[0:4]].values).float()

# Labels তৈরি করো (0,1,2) অনুযায়ী
labels = torch.zeros(len(data), dtype=torch.long)
labels[iris.species == 'versicolor'] = 1
labels[iris.species == 'virginica'] = 2


In [3]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Split data
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2)

# ✅ labels must be LongTensor (for classification)
train_dataset = TensorDataset(torch.tensor(train_data, dtype=torch.float32),
                              torch.tensor(train_labels, dtype=torch.long))

test_dataset = TensorDataset(torch.tensor(test_data, dtype=torch.float32),
                             torch.tensor(test_labels, dtype=torch.long))

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))  # full batch for test


C:\Users\roger\AppData\Local\Temp\ipykernel_5684\2468796170.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(train_data, dtype=torch.float32),
C:\Users\roger\AppData\Local\Temp\ipykernel_5684\2468796170.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(train_labels, dtype=torch.long))
C:\Users\roger\AppData\Local\Temp\ipykernel_5684\2468796170.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_dataset = TensorDataset(torch.tensor(test_data, dtype=torch.float32),
C:\Users\

In [6]:
import torch
import torch.nn as nn

def createANewModel():
    # Model
    ANNiris = nn.Sequential(
        nn.Linear(4, 64),
        nn.ReLU(),
        nn.Linear(64, 64),
        nn.ReLU(),
        nn.Linear(64, 3)
    )
    
    # Loss function
    lossfun = nn.CrossEntropyLoss()

    # Optimizer with L2 regularization (weight_decay)
    optimizer = torch.optim.SGD(ANNiris.parameters(), lr=0.005)

    return ANNiris, lossfun, optimizer


In [8]:
tmpmodel=createANewModel()[0]

print(tmpmodel)

for i in tmpmodel.named_parameters():
    print(i[0],i[1].shape,i[1].numel())

Sequential(
  (0): Linear(in_features=4, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=3, bias=True)
)
0.weight torch.Size([64, 4]) 256
0.bias torch.Size([64]) 64
2.weight torch.Size([64, 64]) 4096
2.bias torch.Size([64]) 64
4.weight torch.Size([3, 64]) 192
4.bias torch.Size([3]) 3


In [ ]:
numepochs = 1000

def trainTheModel():
    trainAcc = []
    testAcc = []
    losses = []

    for epochi in range(numepochs):
        batchAcc = []
        batchLoss = []

        # Training mode
        ANNQC.train()

        for X, y in train_loader:
            ## Forward pass and loss
            yHat = ANNQC(X)
            loss = lossfun(yHat, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accuracy
            preds = torch.argmax(yHat, dim=1)
            acc = 100 * torch.mean((preds == y).float()).item()
            batchAcc.append(acc)
            batchLoss.append(loss.item())

        # Epoch metrics
        trainAcc.append(np.mean(batchAcc))
        losses.append(np.mean(batchLoss))

        # Optional: Test accuracy evaluation per epoch
        ANNQC.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for X, y in test_loader:
                yHat = ANNQC(X)
                preds = torch.argmax(yHat, dim=1)
                correct += (preds == y).sum().item()
                total += y.size(0)
        testAcc.append(100 * correct / total)

        # Optional: Print progress
        if epochi % 100 == 0:
            print(f"Epoch {epochi}: Train Acc = {trainAcc[-1]:.2f}%, Test Acc = {testAcc[-1]:.2f}%, Loss = {losses[-1]:.4f}")

    return trainAcc, testAcc, losses
